# Exploración bayesiana de hiperparámetros

La exploración de hiperparámetros es una de las partes más tediosas y críticas del entrenamiento de redes neuronales. Para obtener resultados que sean correctos, significativos y reproducibles es necesario planificar y sistemizar este proceso de búsqueda.

  >  hyper-parameter optimization should be regarded as a formal outer loop in the
learning process [1]

Formalmente, este proceso se puede describir como la minimización de la función de pérdida (o maximizar la performance) como si fuera una función de *caja negra* que toma como parámetros los valores de los hiperparámetros:


$$ f(\theta) = loss_\theta(y, \hat{y}) $$
$$ \theta^* = argmin_\theta f(\theta) $$

donde $\theta$ es el conjunto de hiperparámetros del modelo, $loss$ es la pérdida generada entre las etiquetas verdaderas $y$ y las etiquetas generadas por el modelo $\hat{y}$, y $f$ es la función objetivo de la minimización.

Los métodos que vimos anteriormente tienen varias desventajas:
  * La exploración por grilla require muchas evaluaciones para lograr cobertura. Además de ello, las combinaciones en dónde sólo se varían hiperparámetros no relevantes no recolectan información nueva.
  * La exploración aleatoria tiene mayor cobertura con menos experimentos, pero realiza combinaciones al azar sin tener en cuenta los resultados obtenidos.
  * La exploración manual es guiada por la intuición de la persona que elige el siguiente conjunto de hiperparámetros, pero requiere de la intervención de un humano antes de cada experimento.

Para solucionar todos estos problemas, es que se utiliza la **exploración bayesiana**. Este método modela la loss como un Gaussian process, y tiene en cuenta los resultados de los experimentos anteriores para ir construyendo una distribución de probabilidad de la pérdida dados los hiperparámetros:

$$ P(loss | \theta)$$

Para elegir una nueva combinación de hiperparámetros a probar dados los experimentos previos, el algoritmo utiliza una *surrogate function* para aproximar el comportamiento de la pérdida y una *selection function* basada en la mejora esperada. A grandes rasgos, el algoritmo sigue los siguientes pasos.


  1. Encontrar el mejor conjunto de hiperparámetros que maximize la mejora esperada (EI), estimada a través de la *surrogate function*.
  2. Calcular la performance del modelo con la combinación de hiperparámetros elegida. Esto corresponde a evaluar la función objetivo.
  3. Actualizar la forma de la *surrogate function* utilizando el teorema de Bayes para que se ajuste mejor a la verdadera distribución $ P(loss | \theta)$

En esta notebook, veremos cómo utilizar la librería `scikit-optimize` o `skopt` para maximizar la performance en función de los hiperparámetros del modelo.

Para más información:

  * [Bayesian optimization with skopt](https://scikit-optimize.github.io/notebooks/bayesian-optimization.html)
  * [Using Bayesian Optimization to reduce the time spent on hyperparameter tuning](https://medium.com/vantageai/bringing-back-the-time-spent-on-hyperparameter-tuning-with-bayesian-optimisation-2e21a3198afb)
  * [1] [Algorithms for Hyper-Parameter Optimization](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)
  * [Practical Bayesian Optimization of Machine Learning Algorithms](https://papers.nips.cc/paper/4522-practical-bayesian-optimization-of-machine-learning-algorithms.pdf)

In [1]:
import os
import numpy
import pandas
import skopt
import seaborn
seaborn.set_style('whitegrid')
seaborn.set_palette('colorblind')
seaborn.set_context('paper')

/users/mteruel/anaconda3/envs/deeplearning/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

from tensorflow.keras import layers, models

## Pre-procesamiento de los datos

Vamos a utilizar los mismos datos y la misma estructura de modelo que la [notebook 2](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/2_data_processing.ipynb). En esta sección, leeremos los datos y extraeremos los features de la misma manera que antes.

In [3]:
DATA_DIRECTORY = '../petfinder_dataset/'

In [4]:
# Take a sample of data

from sklearn.model_selection import train_test_split

dataset, dev_dataset = train_test_split(
    pandas.read_csv(os.path.join(DATA_DIRECTORY, 'train.csv')), test_size=0.2)

In [5]:
target_col = 'AdoptionSpeed'
nlabels = dataset[target_col].unique().shape[0]

In [6]:
# It's important to always use the same one-hot length
one_hot_columns = {
    one_hot_col: dataset[one_hot_col].max()
    for one_hot_col in ['Gender', 'Color1']
}
embedded_columns = {
    embedded_col: dataset[embedded_col].max() + 1
    for embedded_col in ['Breed1']
}
numeric_columns = ['Age', 'Fee']

In [7]:
def process_features(df):
    direct_features = []

    # Create one hot encodings
    for one_hot_col, max_value in one_hot_columns.items():
        direct_features.append(tf.keras.utils.to_categorical(df[one_hot_col] - 1, max_value))

    # Create and append numeric columns
    # Don't forget to normalize!
    # ....
    
    # Concatenate all features that don't need further embedding into a single matrix.
    features = {'direct_features': numpy.hstack(direct_features)}

    # Create embedding columns - nothing to do here. We will use the zero embedding for OOV
    for embedded_col in embedded_columns.keys():
        features[embedded_col] = df[embedded_col].values

    # Convert labels to one-hot encodings
    targets = tf.keras.utils.to_categorical(df[target_col], nlabels)
    
    return features, targets

In [8]:
X_train, y_train = process_features(dataset)
direct_features_input_shape = (X_train['direct_features'].shape[1],)

## Espacio de búsqueda

Para cada hiperparámetro, es necesario definir el rango de valores posibles que puede tomar. Según la [documentación](https://scikit-optimize.github.io/space/space.m.html), existen varios tipos de hiperparámetros posibles:
  * Categorical
  * Integer
  * Real

Para valores Real e Integer, es posible definir una distribución de probabilidad a partir de la cual samplearlos. A modo de ejemplo, agregamos el `learning_rate` del optimizador como parámetro, aunque su impacto no es tan crítico en el optimizador Adam que utilizaremos luego.

In [9]:
from skopt.space import Real, Integer, Categorical

search_space = {
  "batch_size": Integer(10, 100, name="batch_size"),
  "hidden_layer_size": Integer(20, 500, name="hidden_layer_size"),
  "learning_rate": Real(low=1e-4, high=1e-1, prior='log-uniform', name='learning_rate')

}
# Unzipping
search_space_keys, search_space_vals = zip(*search_space.items())
search_space_keys = {param_name: idx
                     for idx, param_name in enumerate(search_space_keys)}
print(search_space)

def hyperparam_value(param_name, param_list):
    return param_list[search_space_keys[param_name]]

def print_selected_hyperparams(param_values):
    for param_name in search_space_keys:
        print("\t", param_name, hyperparam_value(param_name, param_values))

{'batch_size': Integer(low=10, high=100), 'hidden_layer_size': Integer(low=20, high=500), 'learning_rate': Real(low=0.0001, high=0.1, prior='log-uniform', transform='identity')}


## Función objetivo

Lo siguiente que implementaremos es la función objetivo. Es decir, una función que recibe un conjunto de hiperparámetros y devuelve el valor que queremos minimizar. En particular, minimizaremos la loss, pero también podríamos minimizar el accuracy * -1.

Para poder obtener la loss tenemos que
  1. construir el modelo 
  2. entrenarlo en el dataset de train
  3. evaluarlo en el dataset de dev

NOTA: esta función va a utilizar variables desde afuera de su scope, como `X_train` y `y_train`, y si la copian/pegan puede dejar de funcionar.

In [10]:
tf.keras.backend.clear_session()

In [11]:
def objective_function(params):
    print_selected_hyperparams(params)

    batch_size = hyperparam_value("batch_size", params)
    # TODO shuffle the train dataset!
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
    test_ds = tf.data.Dataset.from_tensor_slices(
        process_features(dev_dataset)).batch(batch_size)
    hidden_layer_size = hyperparam_value("hidden_layer_size", params)

    # Add one input and one embedding for each embedded column
    embedding_layers = []
    inputs = []
    for embedded_col, max_value in embedded_columns.items():
        input_layer = layers.Input(shape=(1,), name=embedded_col)
        inputs.append(input_layer)
        # Define the embedding layer
        embedding_size = int(max_value / 4)
        embedding_layers.append(
            tf.squeeze(layers.Embedding(input_dim=max_value, output_dim=embedding_size)(input_layer), axis=-2))
        print('Adding embedding of size {} for layer {}'.format(embedding_size, embedded_col))

    # Add the direct features already calculated
    direct_features_input = layers.Input(shape=direct_features_input_shape, name='direct_features')
    inputs.append(direct_features_input)

    # Concatenate everything together
    features = layers.concatenate(embedding_layers + [direct_features_input])

    dense1 = layers.Dense(hidden_layer_size, activation='relu')(features)
    output_layer = layers.Dense(nlabels, activation='softmax')(dense1)

    # Build model
    learning_rate = hyperparam_value("learning_rate", params)
    model = models.Model(inputs=inputs, outputs=output_layer)
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=['accuracy'])
    
    # Train
    epochs = 10
    model.fit(train_ds, epochs=epochs)
    
    # Evaluate
    loss, accuracy = model.evaluate(test_ds)
    print("*** Test loss: {} - accuracy: {}".format(loss, accuracy))
    
    return loss

## Ejecutar búsqueda


In [12]:
from skopt import gp_minimize

In [16]:
def show_best(res):
    print("Best value: %.4f" % res.fun)
    param_names = {idx: param_name for param_name, idx in search_space_keys.items()}
    best_params = {param_names[i]: param_value for i, param_value in enumerate(res.x)}
    print("Best params:")
    print(best_params)

In [14]:
iterations = 10
exploration_result = gp_minimize(
    objective_function, search_space_vals,
    random_state=21, verbose=1, n_calls=iterations)

Iteration No: 1 started. Evaluating function at random point.
	 batch_size 82
	 hidden_layer_size 346
	 learning_rate 0.00012836393729688734
Adding embedding of size 77 for layer Breed1
Epoch 1/10
147/147 [==============================] - 1s 7ms/step - loss: 1.5500 - accuracy: 0.2595
Epoch 2/10
147/147 [==============================] - 0s 2ms/step - loss: 1.4718 - accuracy: 0.2942
Epoch 3/10
147/147 [==============================] - 0s 2ms/step - loss: 1.4502 - accuracy: 0.3079
Epoch 4/10
147/147 [==============================] - 0s 2ms/step - loss: 1.4444 - accuracy: 0.3152
Epoch 5/10
147/147 [==============================] - 0s 2ms/step - loss: 1.4417 - accuracy: 0.3172
Epoch 6/10
147/147 [==============================] - 0s 2ms/step - loss: 1.4397 - accuracy: 0.3204
Epoch 7/10
147/147 [==============================] - 0s 2ms/step - loss: 1.4380 - accuracy: 0.3216
Epoch 8/10
147/147 [==============================] - 0s 2ms/step - loss: 1.4363 - accuracy: 0.3232
Epoch 9/10
147

462/462 [==============================] - 1s 2ms/step - loss: 1.4306 - accuracy: 0.3277
Epoch 7/10
462/462 [==============================] - 1s 2ms/step - loss: 1.4280 - accuracy: 0.3301
Epoch 8/10
462/462 [==============================] - 1s 2ms/step - loss: 1.4256 - accuracy: 0.3299
Epoch 9/10
462/462 [==============================] - 1s 2ms/step - loss: 1.4234 - accuracy: 0.3305
Epoch 10/10
116/116 [==============================] - 0s 2ms/step - loss: 1.4461 - accuracy: 0.3144
*** Test loss: 1.4460931040089706 - accuracy: 0.314438134431839
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 10.3112
Function value obtained: 1.4461
Current minimum: 1.4433
Iteration No: 5 started. Evaluating function at random point.
	 batch_size 79
	 hidden_layer_size 200
	 learning_rate 0.0018931642317571234
Adding embedding of size 77 for layer Breed1
Epoch 1/10
152/152 [==============================] - 1s 7ms/step - loss: 1.4631 - accuracy: 0.2972
Epoch 2/10
152/152 [=========

135/135 [==============================] - 1s 8ms/step - loss: 1.9747 - accuracy: 0.2741
Epoch 2/10
135/135 [==============================] - 0s 3ms/step - loss: 2.2196 - accuracy: 0.2720
Epoch 3/10
135/135 [==============================] - 0s 3ms/step - loss: 2.3898 - accuracy: 0.2724
Epoch 4/10
135/135 [==============================] - 0s 3ms/step - loss: 2.0356 - accuracy: 0.2748
Epoch 5/10
135/135 [==============================] - 0s 3ms/step - loss: 3.3196 - accuracy: 0.2744
Epoch 6/10
135/135 [==============================] - 0s 3ms/step - loss: 4.8672 - accuracy: 0.2744
Epoch 7/10
135/135 [==============================] - 0s 3ms/step - loss: 1.7897 - accuracy: 0.2767
Epoch 8/10
135/135 [==============================] - 0s 3ms/step - loss: 1.4872 - accuracy: 0.2746
Epoch 9/10
135/135 [==============================] - 0s 3ms/step - loss: 1.7177 - accuracy: 0.2742
Epoch 10/10
34/34 [==============================] - 0s 5ms/step - loss: 3.0022 - accuracy: 0.2578
*** Test los

In [17]:
show_best(exploration_result)

Best value: 1.4433
Best params:
{'batch_size': 82, 'hidden_layer_size': 346, 'learning_rate': 0.00012836393729688734}
